# Convert photos of hydrogels to SEQ file

In [5]:
import os
import rawpy # not a dependency of pynamix - see here for installation instructions: https://github.com/letmaik/rawpy
import skimage.color
import numpy
import tifffile
import pynamix.io

%matplotlib inline

## Load data

In [6]:
data_dir = '/Volumes/PGL/DCIM/102ND750/'
prefix = 'DSC_'
suffix = '.NEF'
start = 1394
stop = start + 71
dtheta = 5
theta = 0

for i,file_number in enumerate(range(start,stop+1)):
    path = data_dir + prefix + str(file_number) + suffix
    with rawpy.imread(path) as raw:
        rgb = raw.postprocess(gamma=(1,1), no_auto_bright=True, output_bps=16)
#         gray = skimage.color.rgb2gray(rgb)
        green = rgb[:,:,1]
        if i == 0:
            nfiles = stop - start + 1
            timeseries = numpy.zeros([nfiles,green.shape[0],green.shape[1]],dtype='uint16')
        timeseries[i] = green
    print(f'Up to theta = {theta}', end='\r')
    theta += dtheta

In [7]:
tifffile.imsave('linear.tiff', timeseries)

In [4]:
logfile = {}
logfile["detector"] = {}
logfile["detector"]["image_size"] = {}
logfile["detector"]["image_size"]["height"] = timeseries.shape[1]
logfile["detector"]["image_size"]["width"] = timeseries.shape[2]
logfile["detector"]["rotate"] = 0 # this is probably wrong
pynamix.io.write_seq('optical_tomograph_test', timeseries, logfile)

In [21]:
?pynamix.io.write_seq

# Communicate with rotating table

In [30]:
# Step one install the library: https://github.com/libusb/hidapi#what-does-the-api-look-like
# Step two install hid for python: https://github.com/apmorton/pyhidapi

import hid

vid = 0x04d8 # About This Mac -> Overview -> System Report -> USB
pid = 0x0404

angle = 10
speed = 900

with hid.Device(vid, pid) as h:
#     print(f'Device manufacturer: {h.manufacturer}')
#     print(f'Product: {h.product}')
    s = f'G91G01X{angle}F{speed};'
    h.write(bytes(bytearray(s.encode())))